In [2]:
# load all necessary libraries
import gensim
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [5]:
# random state for reproducing same results
random_state = 54321

In [8]:
# load data
discourseData = pd.read_csv(r"C:\Masters\Masters in Data Science\Research\EDA\Data\Revised Data\FinalData\Google_Cloud\2HVSingleTextClassification_v2.txt", sep = "\t", names=["label", "message"])
discourseData.head()

,label,message
0,label,message
1,Other,Life is a song - sing it. Life is a game - pla...
2,Love,Life is love - enjoy it.
3,Right_Conduct,"Do not use poisonous words against anyone, for..."
4,Other,The end of education is character


In [9]:
# extract the messages from the dataframe
messages = discourseData.message
labels = discourseData.label
print(len(messages))
print(len(labels))

675
675


In [10]:
# convert messages into list
messages = [message for message in messages]
print(len(messages))

675


In [11]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

# add stemming and lemmatisation in the preprocess function
def preprocess(document, stem=True):
    #changes document to lower case and removes stopwords'

    # change sentence to lower case
    document = document.lower()

    # tokenize into words
    words = word_tokenize(document)

    # remove stop words
    words = [word for word in words if word not in stopwords.words("english") and re.sub(r'\b\w{1,3}\b', '', word)]
    
    if stem:
        words = [stemmer.stem(word) for word in words]
    else:
        words = [wordnet_lemmatizer.lemmatize(word, pos='v') for word in words]

    # join words to make sentence
    document = " ".join(words)
    
    return document

In [12]:
## initialise the inbuilt Stemmer and the Lemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()
import re

In [13]:
# stem messages 'Stem = True'
#Lemma messages  'Stem = False'
messages = [preprocess(message, stem=False) for message in discourseData.message]

In [14]:
print(messages[:2])

['message', 'life song - sing . life game - play . life challenge - meet . life dream - realize . life sacrifice - offer .']


In [15]:
# bag of words model
vectorizer = CountVectorizer()
count_vect = vectorizer.fit_transform(messages)
print(count_vect.shape[0])
# look at the dataframe
print(count_vect.shape[0])
pd.DataFrame(count_vect.toarray(), columns = vectorizer.get_feature_names())
# token names
print(vectorizer.get_feature_names())

675
675
['30', 'aathmashuddhi', 'aberrations', 'abhangs', 'abhayatwam', 'abhimanyu', 'abide', 'abilities', 'ability', 'able', 'abomination', 'abound', 'absence', 'absent', 'absolute', 'absolutely', 'absorb', 'abundant', 'accept', 'acceptable', 'accidents', 'accomplish', 'accomplishments', 'accord', 'accordingly', 'account', 'accumulate', 'acharyadevo', 'achieve', 'achievement', 'achievements', 'acknowledge', 'acquire', 'acquisition', 'across', 'act', 'action', 'active', 'actively', 'activities', 'activity', 'actors', 'actually', 'add', 'addict', 'addition', 'additional', 'address', 'adharma', 'adhere', 'adherence', 'adolescence', 'adopt', 'adoration', 'adore', 'adulthood', 'advancement', 'advantage', 'advent', 'adventure', 'advice', 'advisable', 'advise', 'adviser', 'adwaitaṃ', 'afar', 'affair', 'affairs', 'affect', 'affection', 'affinity', 'affirm', 'afford', 'afraid', 'afterlife', 'afterwards', 'againt', 'age', 'aggravate', 'aggressive', 'agitate', 'agitation', 'ahamkāra', 'ahaṃkāraṃ

In [16]:
## - shuffle the pack to create a random and unbiased split of the dataset
#import random
from sklearn.utils import shuffle
#discourseData = shuffle(discourseData)

In [17]:
## - creating slicing index at 80% threshold
sliceIndex = (int((len(discourseData)*.8)))

In [18]:
trainDF = pd.DataFrame()

In [19]:
trainDF['text'] = discourseData['message']
trainDF['label'] = discourseData['label']

trainDF, testDF = trainDF[:sliceIndex], trainDF[sliceIndex:]

In [20]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'],test_size = 0.20)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

print(len(train_x))
print(len(valid_y))
print(valid_x)

432
108
346    When all follow the sacred path, the world wil...
151    There cannot be life without love. A small exa...
47     Serve others for they are reflections of the s...
347    Give up ignorance. Light the lamp of wisdom wi...
93     God wears Truth, the good seek Truth and the b...
400    Love is no merchandise. Do not bargain about i...
517    What exactly is Truth? Is it a description of ...
216                                         Know thyself
390    Never forget God. God alone is everlasting. Th...
54     A person might be an expert in any field of kn...
416             The entire universe originated from Love
306    Swami says that only through love and not thro...
276    The human being today is behaving with less gr...
180    Love is your life. Love is your goal. The Bhag...
182     Your love becomes worldly in nature when you ...
343    That is the Principle of Love. Worldly love ma...
282    Love is the most essential undercurrent of you...
271    These three path

In [21]:
# create a count vectorizer object - Bag of Words
vectorizer = CountVectorizer()
count_vect = vectorizer.fit(trainDF['text'])


# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [22]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)

In [23]:
print(xtrain_tfidf_ngram_chars)

  (0, 4740)	0.054984373174593024
  (0, 4734)	0.05224471356660047
  (0, 4473)	0.1057132174999689
  (0, 4469)	0.05586881735777562
  (0, 4211)	0.0895499454648524
  (0, 4209)	0.05771750424839857
  (0, 4098)	0.05717792264108226
  (0, 4083)	0.04532896206245029
  (0, 4071)	0.0737103623872937
  (0, 4064)	0.10324067732190599
  (0, 4061)	0.10203579673318519
  (0, 4044)	0.11058241681989457
  (0, 4040)	0.04434254188421
  (0, 3925)	0.15742278742128749
  (0, 3915)	0.14144483805258512
  (0, 3914)	0.1605741576026428
  (0, 3899)	0.09318829424663538
  (0, 3887)	0.06147515349907481
  (0, 3817)	0.09111434361929982
  (0, 3812)	0.07590161616982506
  (0, 3801)	0.08694467225122061
  (0, 3744)	0.07517475294958263
  (0, 3740)	0.055613524486634855
  (0, 3724)	0.06840405352350164
  (0, 3699)	0.11292964136326893
  :	:
  (431, 1085)	0.08559419218992695
  (431, 1078)	0.10164545755336775
  (431, 1075)	0.09738573095708153
  (431, 885)	0.14193695640874948
  (431, 884)	0.13244104737588916
  (431, 751)	0.1192671579118721

In [24]:
print(messages)

['message', 'life song - sing . life game - play . life challenge - meet . life dream - realize . life sacrifice - offer .', 'life love - enjoy .', 'poisonous word anyone , word wind fatally even arrows .', 'education character', 'worldly thoughts diminish , thoughts increase . normally , mind time desire worldly things . desire , peace become stronger', 'caste , caste humanity . religion , religion love . language , language heart .', 'love barren cloud moisture , tree fruit yield milk . ever never earn grace .', 'anger , , jealousy biggest diseases , keep aloof three diseases .', 'life mosaic pleasure pain - grief interval moments .', 'surrender mind completely , take care every .', 'love bridge change . build house .', 'charity mean land full beggar . provide support mean beggar , provide food , clothe conveniences encourage laziness beg .', 'learn speak feel , speak', 'love selflessness , self lovelessness', 'simple sincere', 'be worlds happy . include universal prayer prayers ever

In [ ]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open(r'C:\Users\425858\wiki-news-300d-1M\wiki-news-300d-1M.vec',encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=300)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=300)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

### Machine Learning Algorithms

In [ ]:
# Utility function to train models

def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [ ]:
#Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print ("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("NB, CharLevel Vectors: ", accuracy)

# WordEmbeddings wiki-news-300d-1M.vec
accuracy = train_model(naive_bayes.MultinomialNB(), train_seq_x, train_y, valid_seq_x)
print ("NB, WordEmbeddings Vectors: ", accuracy)




In [ ]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print ("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("LR, CharLevel Vectors: ", accuracy)

# WordEmbeddings wiki-news-300d-1M.vec
accuracy = train_model(linear_model.LogisticRegression(), train_seq_x, train_y, valid_seq_x)
print ("LR, WordEmbeddings Vectors: ", accuracy)


In [ ]:
#SVM Linear
accuracy = train_model(svm.SVC(), xtrain_count, train_y, xvalid_count)
print ("SVM, Count Vectors: ", accuracy)

accuracy = train_model(svm.SVC(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("SVM, WordLevel TF-IDF: ", accuracy)

accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("SVM, N-Gram Vectors: ", accuracy)

accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("SVM, CharLevel Vectors: ", accuracy)

# WordEmbeddings wiki-news-300d-1M.vec
accuracy = train_model(svm.SVC(), train_seq_x, train_y, valid_seq_x)
print ("SVM, WordEmbeddings Vectors: ", accuracy)

In [ ]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(n_estimators=100), xtrain_count, train_y, xvalid_count)
print ("RF, Count Vectors: ", accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(n_estimators=100), xtrain_tfidf, train_y, xvalid_tfidf)
print ("RF, WordLevel TF-IDF: ", accuracy)

# RF on Word Level TF IDF Vectors - n grams
accuracy = train_model(ensemble.RandomForestClassifier(n_estimators=100), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("RF, WordLevel TF-IDF n gram : ", accuracy)

# RF on Character Level TF IDF Vectors - n grams
accuracy = train_model(ensemble.RandomForestClassifier(n_estimators=100), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("RF, CharLevel TF-IDF: ", accuracy)

# WordEmbeddings wiki-news-300d-1M.vec
accuracy = train_model(ensemble.RandomForestClassifier(n_estimators=100), train_seq_x, train_y, valid_seq_x)
print ("RF, WordEmbeddings Vectors: ", accuracy)

In [ ]:
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print ("Xgb, Count Vectors: ", accuracy)

# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print ("Xgb, WordLevel TF-IDF: ", accuracy)

# Extereme Gradient Boosting on Word Level TF IDF Vectors n grams
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram.tocsc(), train_y, xvalid_tfidf_ngram.tocsc())
print ("Xgb, WordLevel n grams TF-IDF: ", accuracy)

# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print ("Xgb, CharLevel Vectors: ", accuracy)

# WordEmbeddings wiki-news-300d-1M.vec
accuracy = train_model(xgboost.XGBClassifier(), train_seq_x, train_y, valid_seq_x)
print ("Xgb, WordEmbeddings Vectors: ", accuracy)

In [ ]:
# kNN on Count Vectors
accuracy = train_model(KNeighborsClassifier(n_neighbors=5), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print ("kNN, Count Vectors: ", accuracy)

# kNN on Word Level TF IDF Vectors
accuracy = train_model(KNeighborsClassifier(n_neighbors=5), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print ("kNN, WordLevel TF-IDF: ", accuracy)

# kNN on Word Level TF IDF Vectors n grams
accuracy = train_model(KNeighborsClassifier(n_neighbors=5), xtrain_tfidf_ngram.tocsc(), train_y, xvalid_tfidf_ngram.tocsc())
print ("kNN, WordLevel n grams TF-IDF: ", accuracy)

# kNN on Character Level TF IDF Vectors
accuracy = train_model(KNeighborsClassifier(n_neighbors=5), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print ("kNN, CharLevel Vectors: ", accuracy)

# WordEmbeddings wiki-news-300d-1M.vec
accuracy = train_model(KNeighborsClassifier(n_neighbors=5), train_seq_x, train_y, valid_seq_x)
print ("kNN, WordEmbeddings Vectors: ", accuracy)


#Finding N for Count Vectors
#https://www.ritchieng.com/machine-learning-k-nearest-neighbors-knn/
# try K=1 through K=25 and record testing accuracy
k_range = range(1, 26)
# We can create Python dictionary using [] or dict()
scores = []
# We use a loop through the range 1 to 26
# We append the scores in the dictionary
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(xtrain_count, train_y)
    y_pred = knn.predict(xvalid_count)
    scores.append(metrics.accuracy_score(valid_y, y_pred))
print("Best N, kNN on Count Vectors : ",scores)

#Finding N for TF-IDF
#https://www.ritchieng.com/machine-learning-k-nearest-neighbors-knn/
# try K=1 through K=25 and record testing accuracy
k_range = range(1, 26)
# We can create Python dictionary using [] or dict()
scores = []
# We use a loop through the range 1 to 26
# We append the scores in the dictionary
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(xtrain_tfidf, train_y)
    y_pred = knn.predict(xvalid_tfidf)
    scores.append(metrics.accuracy_score(valid_y, y_pred))
print("Best N, kNN on TF-IDF : ",scores)

#Finding N for TF-IDF n-grams
#https://www.ritchieng.com/machine-learning-k-nearest-neighbors-knn/
# try K=1 through K=25 and record testing accuracy
k_range = range(1, 26)
# We can create Python dictionary using [] or dict()
scores = []
# We use a loop through the range 1 to 26
# We append the scores in the dictionary
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(xtrain_tfidf_ngram, train_y)
    y_pred = knn.predict(xvalid_tfidf_ngram)
    scores.append(metrics.accuracy_score(valid_y, y_pred))
print("Best N, kNN on TF-IDF n-grams : ",scores)

#Finding N for TF-IDF n-grams Char
#https://www.ritchieng.com/machine-learning-k-nearest-neighbors-knn/
# try K=1 through K=25 and record testing accuracy
k_range = range(1, 26)
# We can create Python dictionary using [] or dict()
scores = []
# We use a loop through the range 1 to 26
# We append the scores in the dictionary
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(xtrain_tfidf_ngram_chars, train_y)
    y_pred = knn.predict(xvalid_tfidf_ngram_chars)
    scores.append(metrics.accuracy_score(valid_y, y_pred))
print("Best N, kNN on TF-IDF n-grams Char : ",scores)


#Finding N for Word Embeddings
#https://www.ritchieng.com/machine-learning-k-nearest-neighbors-knn/
# try K=1 through K=25 and record testing accuracy
k_range = range(1, 26)
# We can create Python dictionary using [] or dict()
scores = []
# We use a loop through the range 1 to 26
# We append the scores in the dictionary
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(train_seq_x, train_y)
    y_pred = knn.predict(valid_seq_x)
    scores.append(metrics.accuracy_score(valid_y, y_pred))
print("Best N, kNN on Word Embeddings : ",scores)



In [ ]:
#Rocchio classification

accuracy = train_model(NearestCentroid(), xtrain_count, train_y, xvalid_count)
print ("Rocchio classification, Count Vectors: ", accuracy)

accuracy = train_model(NearestCentroid(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("Rocchio classification, WordLevel TF-IDF: ", accuracy)

accuracy = train_model(NearestCentroid(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("Rocchio classification, N-Gram Vectors: ", accuracy)

accuracy = train_model(NearestCentroid(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("Rocchio classification, CharLevel Vectors: ", accuracy)

# WordEmbeddings wiki-news-300d-1M.vec
accuracy = train_model(NearestCentroid(), train_seq_x, train_y, valid_seq_x)
print ("Rocchio classification, WordEmbeddings Vectors: ", accuracy)

In [ ]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

accuracy = train_model(GradientBoostingClassifier(n_estimators=100), xtrain_count, train_y, xvalid_count)
print ("Gradient Boosting, Count Vectors: ", accuracy)

accuracy = train_model(GradientBoostingClassifier(n_estimators=100), xtrain_tfidf, train_y, xvalid_tfidf)
print ("Gradient Boosting, WordLevel TF-IDF: ", accuracy)

accuracy = train_model(GradientBoostingClassifier(n_estimators=100), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("Gradient Boostingn, N-Gram Vectors: ", accuracy)

accuracy = train_model(GradientBoostingClassifier(n_estimators=100), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("Gradient Boosting, CharLevel Vectors: ", accuracy)

# WordEmbeddings wiki-news-300d-1M.vec
accuracy = train_model(GradientBoostingClassifier(n_estimators=100), train_seq_x, train_y, valid_seq_x)
print ("Gradient Boosting, WordEmbeddings Vectors: ", accuracy)

In [ ]:
##Meta Bagging Estimator with KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

accuracy = train_model(BaggingClassifier(KNeighborsClassifier(n_neighbors=4)), xtrain_count, train_y, xvalid_count)
print ("Bagging-K Nearest, Count Vectors: ", accuracy)

accuracy = train_model(BaggingClassifier(KNeighborsClassifier(n_neighbors=3)), xtrain_tfidf, train_y, xvalid_tfidf)
print ("Bagging-K Nearest, WordLevel TF-IDF: ", accuracy)

accuracy = train_model(BaggingClassifier(KNeighborsClassifier(n_neighbors=16)), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("Bagging-K Nearest, N-Gram Vectors: ", accuracy)

accuracy = train_model(BaggingClassifier(KNeighborsClassifier(n_neighbors=3)), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("Bagging-K Nearest, CharLevel Vectors: ", accuracy)

# WordEmbeddings wiki-news-300d-1M.vec
accuracy = train_model(BaggingClassifier(KNeighborsClassifier(n_neighbors=13)), train_seq_x, train_y, valid_seq_x)
print ("Bagging-K Nearest, WordEmbeddings Vectors: ", accuracy)

In [ ]:
#Meta Bagging Decision Trees
from sklearn import tree
accuracy = train_model(BaggingClassifier(tree.DecisionTreeClassifier()), xtrain_count, train_y, xvalid_count)
print ("Decision Trees, Count Vectors: ", accuracy)

accuracy = train_model(BaggingClassifier(tree.DecisionTreeClassifier()), xtrain_tfidf, train_y, xvalid_tfidf)
print ("Decision Trees, WordLevel TF-IDF: ", accuracy)

accuracy = train_model(BaggingClassifier(tree.DecisionTreeClassifier()), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("Decision Trees, N-Gram Vectors: ", accuracy)

accuracy = train_model(BaggingClassifier(tree.DecisionTreeClassifier()), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("Decision Trees, CharLevel Vectors: ", accuracy)

# WordEmbeddings wiki-news-300d-1M.vec
accuracy = train_model(BaggingClassifier(tree.DecisionTreeClassifier()), train_seq_x, train_y, valid_seq_x)
print ("Decision Trees, WordEmbeddings Vectors: ", accuracy)

In [ ]:
#https://jakevdp.github.io/PythonDataScienceHandbook/05.07-support-vector-machines.html
#kernel = linear
#kernel='poly', degree=8
#kernel = rbf - gives best results
#kernel = sigmoid

accuracy = train_model(svm.SVC(kernel='rbf'), xtrain_count, train_y, xvalid_count)
print ("Count Vectors: ", accuracy)

accuracy = train_model(svm.SVC(kernel='rbf'), xtrain_tfidf, train_y, xvalid_tfidf)
print ("SVM, WordLevel TF-IDF: ", accuracy)

accuracy = train_model(svm.SVC(kernel='rbf'), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("SVM, N-Gram Vectors: ", accuracy)

accuracy = train_model(svm.SVC(kernel='rbf'), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("CharLevel Vectors: ", accuracy)

# WordEmbeddings wiki-news-300d-1M.vec
accuracy = train_model(svm.SVC(kernel='rbf'), train_seq_x, train_y, valid_seq_x)
print ("SVM, WordEmbeddings Vectors: ", accuracy)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

accuracy = train_model(AdaBoostClassifier(random_state=1), xtrain_count, train_y, xvalid_count)
print ("AdaBoost Classifier, Count Vectors: ", accuracy)

accuracy = train_model(AdaBoostClassifier(random_state=1), xtrain_tfidf, train_y, xvalid_tfidf)
print ("AdaBoost Classifier , WordLevel TF-IDF: ", accuracy)

accuracy = train_model(AdaBoostClassifier(random_state=1), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("AdaBoost Classifier , N-Gram Vectors: ", accuracy)

accuracy = train_model(AdaBoostClassifier(random_state=1), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("AdaBoost Classifier , CharLevel Vectors: ", accuracy)

# WordEmbeddings wiki-news-300d-1M.vec
accuracy = train_model(AdaBoostClassifier(random_state=1), train_seq_x, train_y, valid_seq_x)
print ("AdaBoost Classifier, WordEmbeddings Vectors: ", accuracy)

In [ ]:
#Meta Bagging Estimator with Decision Trees
from sklearn.ensemble import BaggingClassifier
from sklearn import tree

accuracy = train_model(BaggingClassifier(tree.DecisionTreeClassifier(random_state=1)), xtrain_count, train_y, xvalid_count)
print ("Meta Bagging Estimator with Decision Trees, Count Vectors: ", accuracy)

accuracy = train_model(BaggingClassifier(tree.DecisionTreeClassifier(random_state=1)), xtrain_tfidf, train_y, xvalid_tfidf)
print ("Meta Bagging Estimator with Decision Trees, WordLevel TF-IDF: ", accuracy)

accuracy = train_model(BaggingClassifier(tree.DecisionTreeClassifier(random_state=1)), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("Meta Bagging Estimator with Decision Trees, N-Gram Vectors: ", accuracy)

accuracy = train_model(BaggingClassifier(tree.DecisionTreeClassifier(random_state=1)), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("Meta Bagging Estimator with Decision Trees, CharLevel Vectors: ", accuracy)

# WordEmbeddings wiki-news-300d-1M.vec
accuracy = train_model(BaggingClassifier(tree.DecisionTreeClassifier(random_state=1)), train_seq_x, train_y, valid_seq_x)
print ("Meta Bagging Estimator with Decision Trees, WordEmbeddings Vectors: ", accuracy)

### Code to run all algorithms

In [ ]:
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn import decomposition, ensemble
from keras import layers, models, optimizers
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
import numpy as np
import pandas as pd
import seaborn as sns; sns.set(style="ticks", color_codes=True)
import warnings; warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import time

In [ ]:
### CONFIGURING THE MODELS TO TRAIN ###

dict_classifiers = {
    "Logistic Regression": linear_model.LogisticRegression(),
    "Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
    "Decision Tree": tree.DecisionTreeClassifier(),
    "Random Forest": ensemble.RandomForestClassifier(max_depth=8,n_estimators=100),    
    #"Naive Bayes": GaussianNB(),
    "Support Vector  - Linear": svm.SVC(),
    "Naive Bayes Mutinomial": naive_bayes.MultinomialNB(),
    "Extreme Gradient Boosting": xgboost.XGBClassifier(),
    "Rocchio classification": NearestCentroid(),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100),
    "Meta Bagging Estimator with KNeighborsClassifier": BaggingClassifier(KNeighborsClassifier(n_neighbors=5)),
    "Meta Bagging Estimator with Decision Trees": BaggingClassifier(tree.DecisionTreeClassifier(random_state=1)),
    "SVM - rbf": svm.SVC(kernel='rbf'),
    "Ada boosting": AdaBoostClassifier(random_state=1)
    }

In [ ]:
###### ACTUAL MODEL TRAINING LOGIC ########

def bulk_train(X_train, Y_train, X_test, Y_test, no_classifiers = 13, verbose = True):
    model_list_pickle = {}
    
    for classifier_name, classifier in list(dict_classifiers.items())[:no_classifiers]:
        start_time = time.clock()
        if verbose:
            print("Training {c} - Start ".format(c=classifier_name))       
        classifier.fit(X_train, np.ravel(Y_train,order='C'))
        end_time = time.clock()        
        total_time_to_train = end_time - start_time
        train_score = classifier.score(X_train, Y_train)
        test_score = classifier.score(X_test, Y_test)
        if verbose:
            print(train_score, test_score)
        
        ############# Also GOING TO CROSS VALIDATE ##############
        from sklearn.model_selection import cross_val_score
        accuracies_cv= cross_val_score(estimator = classifier, X = X_train, y = Y_train, cv = 10)
        accuracies_cv_mean=accuracies_cv.mean()*100
        print("%s : Mean Accuracy from CV is %s %%"%(classifier_name,accuracies_cv_mean))
        accuracies_cv_std=accuracies_cv.std()*100
        print("%s : Standard Deviation from CV %s %%"%(classifier_name,accuracies_cv_std))
        ################# END OF CROSS VALIDATE #################
        
        model_list_pickle[classifier_name] = {'model': classifier, 
                                              'train_score': train_score, 
                                              'test_score': test_score, 
                                              'train_time': total_time_to_train,
                                              'crossval_avg_accuracy': accuracies_cv_mean,
                                              'crossval_std': accuracies_cv_std
                                             }
        
        if verbose:
            print("trained {c} in {f:.2f} s".format(c=classifier_name, f=total_time_to_train))
            
    return model_list_pickle

In [ ]:
#### FUNCTION TO RECORD MODEL METRICS AS DATAFRAME ###

def print_confusion_matrix(model_list, X_test):
    cls_key = [key for key in model_list.keys()]
    cm_df_values = [[]]
    cm_df_cols = ['classifer', 'True_Positive', 'True_Negative', 'False_Postive', 'False_Negative',
                  'Train_AccuracyScore','Test_AccuracyScore', 'CrossVal_Accuracy%', 'CrossVal_STD']
    for i in range(0, len(cls_key)):
        predictions = model_list[cls_key[i]]['model'].predict(X_test)
        cm = confusion_matrix(y_test, predictions)
        cr = classification_report(y_test, predictions)
        print("\n###### Confusion Matrix Summary for [%s] ######\n"%(cls_key[i]))
        print(cm)
        print("\n###### Classification Report for [%s] ######\n"%(cls_key[i]))
        print(cr)
        if (i == 0) :
            cm_df_values = [[cls_key[i], cm[1,1], cm[0,0], cm[0,1], cm[1,0],
                            model_list[cls_key[i]]['train_score'], model_list[cls_key[i]]['test_score'],
                            model_list[cls_key[i]]['crossval_avg_accuracy'], model_list[cls_key[i]]['crossval_std']]]
        else :            
            cm_df_values = cm_df_values + [[cls_key[i], cm[1,1], cm[0,0], cm[0,1], cm[1,0],
                                           model_list[cls_key[i]]['train_score'], model_list[cls_key[i]]['test_score'],
                                           model_list[cls_key[i]]['crossval_avg_accuracy'], model_list[cls_key[i]]['crossval_std']]]
       
    cm_df = pd.DataFrame(data=cm_df_values, columns=cm_df_cols)
    print("\n###### Exiting Confusion Matrix Summary ######\n")
    return cm_df

#TN (0,0), FP (0,1), FN (1,0), TP (1,1)

In [ ]:
### METHOD TO SUMMARIZE THE MODEL TRAINING i.e. just to PRINT THE OUTCOME ###

def log_train_summary(model_list, sort_by='test_score'):
    cls_key = [key for key in model_list.keys()]
    test_local = [model_list[key]['test_score'] for key in cls_key]
    training_local = [model_list[key]['train_score'] for key in cls_key]
    training_time_local = [model_list[key]['train_time'] for key in cls_key]
    
    df_ = pd.DataFrame(data=np.zeros(shape=(len(cls_key),4)), columns = ['classifier', 'train_score', 'test_score', 'train_time'])
    for i in range(0,len(cls_key)):
        df_.loc[i, 'classifier'] = cls_key[i]
        df_.loc[i, 'train_score'] = training_local[i]
        df_.loc[i, 'test_score'] = test_local[i]
        df_.loc[i, 'train_time'] = training_time_local[i]
    
    #overriding the sort clause passed in function to get list of sort criteria for now
    print(df_.sort_values(by=['test_score','train_score'], ascending=False))

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
def generate_auc_roc_curve(clf, X_test):
    y_pred_proba = clf.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test,  y_pred_proba)
    auc = roc_auc_score(y_test, y_pred_proba)
    plt.plot(fpr,tpr,label="AUC ROC Curve with Area Under the curve ="+str(auc))
    plt.legend(loc=4)
    plt.show()
    pass

### Voting Classifier tfidf_vect_ngram_chars

In [ ]:
#https://www.kaggle.com/den3b81/better-predictions-stacking-with-votingclassifier
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier 
from xgboost import XGBClassifier

In [ ]:
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state = 54321)

In [ ]:
# random state for reproducing same results
random_state = 54321
X_train, X_val, y_train, y_val = model_selection.train_test_split(trainDF['text'], trainDF['label'],test_size = 0.20)
#Do not split, use earlier created datasets

In [ ]:
#Soft Voting
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_val = encoder.fit_transform(y_val)

In [ ]:
# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
X_train =  tfidf_vect_ngram_chars.transform(X_train) 
X_val =  tfidf_vect_ngram_chars.transform(X_val)

In [ ]:
#we will combine the predictions of 4 different classifiers, trained with basic parametrization. Tuning the parameters is beyond the scope of this notebook.
#The classifiers are: 1) RandomForestClassifier with "entropy" criterion 2) RandomForestClassifier with "gini" criterion 3) Sklearn GradientBoostingClassifier 4) XGBoost classifier

rf1 = BaggingClassifier(tree.DecisionTreeClassifier())
#rf1 = RandomForestClassifier(n_estimators=1000, criterion='entropy',  n_jobs = -1,  random_state=random_state)
rf1.fit(X_train, y_train)
y_val_pred = rf1.predict_proba(X_val)
log_loss(y_val, y_val_pred)

In [ ]:
rf2= RandomForestClassifier(n_estimators=1000, criterion='gini',  n_jobs = -1, random_state=random_state)
rf2.fit(X_train, y_train)
y_val_pred = rf2.predict_proba(X_val)
log_loss(y_val, y_val_pred)

In [ ]:
#gbc = GradientBoostingClassifier(random_state=random_state,n_estimators=1000)
gbc = GradientBoostingClassifier(n_estimators=1000)
gbc.fit(X_train, y_train)
y_val_pred = gbc.predict_proba(X_val)
log_loss(y_val, y_val_pred)

In [ ]:
xgb = XGBClassifier(seed=random_state,n_estimators=1000)
xgb.fit(X_train, y_train)
y_val_pred = xgb.predict_proba(X_val)
log_loss(y_val, y_val_pred)

In [ ]:
eclf = VotingClassifier(estimators=[
    ('rf1', rf1), ('rf2', rf2), ('gbc', gbc), ('xgb',xgb)], voting='soft')
eclf.fit(X_train, y_train)
y_val_pred = eclf.predict_proba(X_val)
log_loss(y_val, y_val_pred)

In [ ]:
eclf = VotingClassifier(estimators=[
    ('rf1', rf1), ('rf2', rf2), ('gbc', gbc), ('xgb',xgb)], voting='soft', weights = [1,1,3,1])
eclf.fit(X_train, y_train)
y_val_pred = eclf.predict_proba(X_val)
log_loss(y_val, y_val_pred)

In [ ]:
#https://stats.stackexchange.com/questions/349540/hard-voting-soft-voting-in-ensemble-based-methods
#http://rasbt.github.io/mlxtend/user_guide/classifier/EnsembleVoteClassifier/

### Training with different Models

In [ ]:
### Training with Different Models - Logistics Regression, KNN, NB, SVM, Random Forest and Decision Tree
#Dont run, generating errors
models_list = bulk_train(xtrain_count, train_y, xvalid_count, valid_y, no_classifiers = 13, verbose=False)
print("\n")
print("############ CONSOLIDATED MODEL TRAINING SUMMARY - Count Vectors ############")
print("############### ######################## ###################")
log_train_summary(models_list)
print("############### ######################## ###################")

### End of Code 1
